<a href="https://colab.research.google.com/github/drfperez/utilities/blob/main/PDFCompressor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install -y -qq ghostscript

import os
import time
import subprocess
from google.colab import files
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# ---------------- Upload PDF ----------------
print("Upload a PDF file to compress:")
uploaded = files.upload()
if not uploaded:
    raise SystemExit("No PDF uploaded.")

pdf_file = list(uploaded.keys())[0]
print(f"Uploaded: {pdf_file}")

# ---------------- Widgets ----------------
target_size = widgets.FloatSlider(
    value=1.0,
    min=0.1,
    max=50.0,
    step=0.1,
    description="Target Size (MB):",
    style={'description_width': 'initial'}
)

run_btn = widgets.Button(description="Compress PDF", button_style="success")
out = widgets.Output()
display(target_size, run_btn, out)

# ---------------- Helper ----------------
def download_link(path):
    display(HTML(f'<a href="{path}" download="{path}">Download {path}</a>'))

# ---------------- GS Compress ----------------
def compress_pdf_gs(input_pdf, output_pdf, res):
    cmd = [
        'gs', '-sDEVICE=pdfwrite', '-dCompatibilityLevel=1.4',
        '-dPDFSETTINGS=/default',
        '-dNOPAUSE', '-dQUIET', '-dBATCH',
        '-sOutputFile=' + output_pdf,
        '-dCompressFonts=true', '-dSubsetFonts=true',
        '-dDownsampleColorImages=true', '-dColorImageResolution=' + str(res),
        '-dDownsampleGrayImages=true', '-dGrayImageResolution=' + str(res),
        '-dDownsampleMonoImages=true', '-dMonoImageResolution=' + str(res),
        '-dColorImageDownsampleType=/Bicubic',
        '-dGrayImageDownsampleType=/Bicubic',
        '-dMonoImageDownsampleType=/Bicubic',
        input_pdf
    ]
    subprocess.call(cmd)

# ---------------- Auto Compress ----------------
def compress_pdf_auto(input_pdf, output_pdf, target_mb):
    original_size = os.path.getsize(input_pdf) / (1024 * 1024)
    print(f"Original size: {original_size:.2f} MB | Target: {target_mb:.2f} MB")

    # If original is already smaller or equal, just copy it
    if original_size <= target_mb:
        print("Original size is already under target. Copying file.")
        subprocess.call(['cp', input_pdf, output_pdf])
        return output_pdf

    # Binary search on resolution
    low, high = 50, 300
    best_res = high
    best_size = original_size
    temp_file = "temp.pdf"

    while low <= high:
        mid = (low + high) // 2
        compress_pdf_gs(input_pdf, temp_file, mid)
        size_mb = os.path.getsize(temp_file) / (1024 * 1024) if os.path.exists(temp_file) else float('inf')
        print(f"  Resolution {mid} → {size_mb:.2f} MB")

        if size_mb <= target_mb:
            best_res = mid
            best_size = size_mb
            low = mid + 1  # Try higher resolution (better quality)
        else:
            high = mid - 1  # Reduce resolution

    if best_res == 300 and best_size > target_mb:
        print("Could not reach target even at lowest resolution. Using resolution 50.")
        best_res = 50

    # Final compression with best resolution
    print(f"Best resolution: {best_res} → {best_size:.2f} MB (approx)")
    compress_pdf_gs(input_pdf, output_pdf, best_res)

    final_size = os.path.getsize(output_pdf) / (1024 * 1024)
    print(f"Final compressed size: {final_size:.2f} MB")

    # Clean up
    if os.path.exists(temp_file):
        os.remove(temp_file)

    return output_pdf

# ---------------- Run Action ----------------
def run_compress(b):
    with out:
        clear_output(wait=True)
        ts = int(time.time())
        output_file = f"{os.path.splitext(pdf_file)[0]}_compressed_{ts}.pdf"
        compress_pdf_auto(pdf_file, output_file, target_size.value)
        print("Compression done!")
        download_link(output_file)

run_btn.on_click(run_compress)

Upload a PDF file to compress:


Saving Vector Mechanics for Engineers_ Statics & Dynamics, 12th -- Ferdinand P_ Beer & E_ Russell Johnston Jr & David F_ -- Twelfth edition, New York, NY, -- 9781259638091 -- 4eaa122f379c9badd138d1fa53486072 -- Anna’s Archive.pdf to Vector Mechanics for Engineers_ Statics & Dynamics, 12th -- Ferdinand P_ Beer & E_ Russell Johnston Jr & David F_ -- Twelfth edition, New York, NY, -- 9781259638091 -- 4eaa122f379c9badd138d1fa53486072 -- Anna’s Archive.pdf
Uploaded: Vector Mechanics for Engineers_ Statics & Dynamics, 12th -- Ferdinand P_ Beer & E_ Russell Johnston Jr & David F_ -- Twelfth edition, New York, NY, -- 9781259638091 -- 4eaa122f379c9badd138d1fa53486072 -- Anna’s Archive.pdf


FloatSlider(value=1.0, description='Target Size (MB):', max=50.0, min=0.1, style=SliderStyle(description_width…

Button(button_style='success', description='Compress PDF', style=ButtonStyle())

Output()